[View in Colaboratory](https://colab.research.google.com/github/pablomoraisandrade/Kaggle-avito/blob/master/Kaggle_Avito_Download_dataset.ipynb)

API Credentials
To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (https://www.kaggle.com//account) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials.

Place this file on your Google Drive anywhere.

With the next snippet you download your credentials to Colab and you can start using Kaggle API

In [1]:
!ls

datalab


In [2]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [3]:
!pip install kaggle

  Running setup.py bdist_wheel for kaggle ... - done
  Stored in directory: /content/.cache/pip/wheels/d6/3c/92/375decc0cf0a65b281db3cca9b8e21dc27b8ebc4d1ce29afca
Successfully built kaggle


In [4]:
!kaggle competitions list

ref                                             deadline             category            reward  teamCount  userHasEntered  
----------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
imagenet-object-detection-challenge             2029-12-31 07:00:00  Research         Knowledge          0           False  
imagenet-object-detection-from-video-challenge  2029-12-31 07:00:00  Research         Knowledge          0           False  
imagenet-object-localization-challenge          2029-12-31 07:00:00  Research         Knowledge          8           False  
titanic                                         2020-04-07 00:00:00  Getting Started  Knowledge      11321            True  
house-prices-advanced-regression-techniques     2020-03-01 23:59:00  Getting Started  Knowledge       5393           False  
digit-recognizer                                2020-01-07 00:00:00  Getting Started  Knowledge       2504           False  


In [5]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         359G  5.7G  335G   2% /
tmpfs           6.4G     0  6.4G   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
/dev/root       1.2G  537M  684M  44% /opt/bin
tmpfs           6.4G  248M  6.2G   4% /usr/lib64-nvidia
/dev/sda1       365G  7.1G  358G   2% /etc/hosts
shm              64M     0   64M   0% /dev/shm
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [0]:
! mkdir kaggle-avito-data

In [7]:
!kaggle competitions list

ref                                             deadline             category            reward  teamCount  userHasEntered  
----------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
imagenet-object-detection-challenge             2029-12-31 07:00:00  Research         Knowledge          0           False  
imagenet-object-detection-from-video-challenge  2029-12-31 07:00:00  Research         Knowledge          0           False  
imagenet-object-localization-challenge          2029-12-31 07:00:00  Research         Knowledge          8           False  
titanic                                         2020-04-07 00:00:00  Getting Started  Knowledge      11321            True  
house-prices-advanced-regression-techniques     2020-03-01 23:59:00  Getting Started  Knowledge       5393           False  
digit-recognizer                                2020-01-07 00:00:00  Getting Started  Knowledge       2504           False  


In [8]:
!kaggle competitions download -c avito-demand-prediction -p kaggle-avito-data

sample_submission.csv: Downloaded 8MB of 8MB to kaggle-avito-data


test_jpg.zip: Downloaded 19GB of 19GB to kaggle-avito-data


train_jpg.zip: Downloaded 49GB of 49GB to kaggle-avito-data
test.csv.zip: Downloaded 107MB of 107MB to kaggle-avito-data
periods_test.csv.zip: Downloaded 136MB of 136MB to kaggle-avito-data
periods_train.csv.zip: Downloaded 170MB of 170MB to kaggle-avito-data
train.csv.zip: Downloaded 308MB of 308MB to kaggle-avito-data


test_active.csv.zip: Downloaded 2GB of 2GB to kaggle-avito-data
train_active.csv.zip: Downloaded 3GB of 3GB to kaggle-avito-data


train_jpg_4.zip: Downloaded 10GB of 10GB to kaggle-avito-data
train_jpg_2.zip: Downloaded 10GB of 10GB to kaggle-avito-data


train_jpg_1.zip: Downloaded 10GB of 10GB to kaggle-avito-data
train_jpg_0.zip: Downloaded 10GB of 10GB to kaggle-avito-data


train_jpg_3.zip: Downloaded 10GB of 10GB to kaggle-avito-data


In [9]:
! ls -ltrh kaggle-avito-data/

total 124G
-rw-r--r-- 1 root root 8.3M May 19 21:02 sample_submission.csv
-rw-r--r-- 1 root root  19G May 19 21:05 test_jpg.zip
-rw-r--r-- 1 root root  50G May 19 21:11 train_jpg.zip
-rw-r--r-- 1 root root 108M May 19 21:11 test.csv.zip
-rw-r--r-- 1 root root 137M May 19 21:11 periods_test.csv.zip
-rw-r--r-- 1 root root 171M May 19 21:11 periods_train.csv.zip
-rw-r--r-- 1 root root 308M May 19 21:11 train.csv.zip
-rw-r--r-- 1 root root 2.4G May 19 21:11 test_active.csv.zip
-rw-r--r-- 1 root root 2.6G May 19 21:11 train_active.csv.zip
-rw-r--r-- 1 root root  10G May 19 21:13 train_jpg_4.zip
-rw-r--r-- 1 root root 9.9G May 19 21:14 train_jpg_2.zip
-rw-r--r-- 1 root root  10G May 19 21:15 train_jpg_1.zip
-rw-r--r-- 1 root root  10G May 19 21:16 train_jpg_0.zip
-rw-r--r-- 1 root root  10G May 19 21:17 train_jpg_3.zip


In [10]:
! unzip kaggle-avito-data/test.csv.zip

Archive:  kaggle-avito-data/test.csv.zip
  inflating: test.csv                


In [11]:
! unzip kaggle-avito-data/periods_test.csv.zip

Archive:  kaggle-avito-data/periods_test.csv.zip
  inflating: periods_test.csv        


In [12]:
! unzip kaggle-avito-data/periods_train.csv.zip

Archive:  kaggle-avito-data/periods_train.csv.zip
  inflating: periods_train.csv       


In [13]:
! unzip kaggle-avito-data/train.csv.zip

Archive:  kaggle-avito-data/train.csv.zip
  inflating: train.csv               


In [14]:
! ls -ltrh

total 2.5G
-rw-rw-r-- 1 root root 909M Apr 23 16:43 train.csv
-rw-rw-r-- 1 root root 316M Apr 23 16:54 test.csv
-rw-rw-r-- 1 root root 733M Apr 23 17:00 periods_train.csv
-rw-rw-r-- 1 root root 603M Apr 23 17:06 periods_test.csv
drwxr-xr-x 1 root root 4.0K May 19 21:02 datalab
drwxr-xr-x 2 root root 4.0K May 19 21:16 kaggle-avito-data


In [15]:
!head -5 train.csv

item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,,,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяца.цвет серый",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c7679f17c333c959b19,1008.0,0.12789
2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,,,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d71427001e1e0859c0fb843a31,692.0,0.0
ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",,,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ray, USB. Если настроить, то работает смарт тв 

In [16]:
!head -5 test.csv

item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
6544e41a8817,dbe73ad6e4b5,Волгоградская область,Волгоград,Личные вещи,Детская одежда и обувь,Для мальчиков,Обувь,25,Отдам бесплатно,На ангарском,,66,2017-04-18,Private,a8b57acb5ab304f9c331ac7a074219aed4d349d8aef386bd5e32262735e68512,2020.0
65b9484d670f,2e11806abe57,Свердловская область,Нижняя Тура,Хобби и отдых,Велосипеды,Дорожные,,,Продам велосипед,"Продам велосипед KAMA  F200,в нормальном состоянии.Все вопросы по телефону.торг уместен.",3000.0,4,2017-04-16,Private,,
8bab230b2ecd,0b850bbebb10,Новосибирская область,Бердск,Бытовая электроника,Аудио и видео,Телевизоры и проекторы,,,BBK,Продам новый телевизор BBK  32 диагональ смарт ТВ Wi-Fi приложения на базе андроида продаю в связи с покупкой более большего размера,15000.0,15,2017-04-17,Private,8c361112cb049745ef2d1b0ae73594fc5c107286b0c9428bf86da6d04eb6b7d5,2960.0
8e3

In [17]:
!pip install xgboost

In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train = pd.read_csv('train.csv')
del train['image']
del train['image_top_1']
del train['user_id']
target = 'deal_probability'
IDcol = 'item_id'

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
train

,item_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,deal_probability
0,b912c3c6a6ad,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,0.12789
1,2dac0150717d,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,0.00000
2,ba83aefab5dc,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,0.43177
3,02996f1dd2ea,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,0.80323
4,7c90be56d2ab,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,0.20797
5,51e0962387f7,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,0.80323
6,c4f260a2b48a,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0.00000
7,6b71309d6a8a,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,0.80323
8,c5b969cb63a2,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,0.00000
9,b1570962e68c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,0.00000


In [0]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, show_progress=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print( "\nModel Report" )
    print( "Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions) )
    print( "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob) )
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')


In [6]:
target

'deal_probability'

In [0]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

In [8]:
xgb1

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=4, objective='reg:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [9]:
train.columns

Index(['item_id', 'region', 'city', 'parent_category_name', 'category_name',
       'param_1', 'param_2', 'param_3', 'title', 'description', 'price',
       'item_seq_number', 'activation_date', 'user_type', 'deal_probability'],
      dtype='object')

In [10]:
predictors

['region',
 'city',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'title',
 'description',
 'price',
 'item_seq_number',
 'activation_date',
 'user_type']

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
label_encoder = LabelEncoder()
train['region'] = label_encoder.fit_transform(train['region'].astype('str'))
train['city'] = label_encoder.fit_transform(train['city'].astype('str'))
train['parent_category_name'] = label_encoder.fit_transform(train['parent_category_name'].astype('str'))
train['category_name'] = label_encoder.fit_transform(train['category_name'].astype('str'))
train['param_1'] = label_encoder.fit_transform(train['param_1'].astype('str'))
train['param_2'] = label_encoder.fit_transform(train['param_2'].astype('str'))
train['param_3'] = label_encoder.fit_transform(train['param_3'].astype('str'))
train['title'] = label_encoder.fit_transform(train['title'].astype('str'))
train['description'] = label_encoder.fit_transform(train['description'].astype('str'))
train['activation_date'] = label_encoder.fit_transform(train['activation_date'].astype('str'))
train['user_type'] = label_encoder.fit_transform(train['user_type'].astype('str'))


In [0]:
#modelfit(xgb1, train, predictors)
alg=xgb1
dtrain=train
useTrainCV=True
cv_folds=5
early_stopping_rounds=50
if useTrainCV:
    xgb_param = alg.get_xgb_params()
    xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
    cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
        metrics='auc', early_stopping_rounds=early_stopping_rounds)
    alg.set_params(n_estimators=cvresult.shape[0])

#Fit the algorithm on the data
alg.fit(dtrain[predictors], dtrain['deal_probability'],eval_metric='auc')

#Predict training set:
dtrain_predictions = alg.predict(dtrain[predictors])
dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

#Print model report:
print( "\nModel Report" )
print( "Accuracy : %.4g" % metrics.accuracy_score(dtrain['deal_probability'].values, dtrain_predictions) )
print( "AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['deal_probability'], dtrain_predprob) )

feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')